In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display,Markdown
import json
from PyPDF2 import PdfReader
import gradio as gr
from pydantic import BaseModel
import requests


In [4]:
load_dotenv(override=True)

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
gemini_client = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai")
sendgrid_client = os.getenv("SENDGRID_API_KEY")
sendgrid_user = os.getenv("PUSHOVER_USER")
sendgrid_token = os.getenv("PUSHOVER_TOKEN")


In [5]:
def Pushover(message):
    url = "https://api.pushover.net/1/messages.json"
    payload = {
        "token": sendgrid_token,
        "user": sendgrid_user,
        "message": message
    }
    response = requests.post(url, data=payload)
    return response.json()

In [6]:
def record_user_details(email, name="Not set", note="Not set"):
    Pushover(f"User {name} with email {email} has been recorded with note {note}")
    print(f"User {name} with email {email} has been recorded with note {note}")
    return {"status": "success"}

In [7]:
def record_unknown_question(question):
    Pushover(f"Unknown question: {question}")
    print(f"Unknown question: {question}")
    return {"status": "success"}

In [8]:
record_unknown_question_tool_structure = {
    "name": "record_unknown_question",
    "description": "Use this tool to record an unknown question asked by the user",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to record"
            }
        }, 
        "required": ["question"]
    }
}

In [9]:
record_user_details_tool_structure = {
    "name": "record_user_details",
    "description": "Use this tool to record the details of the user that is interested in getting in touch",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email of the user"
            },
            "name": {
                "type": "string",
                "description": "The name of the user"
            },
            "note": {
                "type": "string",
                "description": "The note of the user"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [38]:
def handle_tool_call(tool_calls):
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        print(f"Tool Called: {tool_name} with args: {tool_args}")
        tool = globals()[tool_name]
        tool_result = tool(**tool_args) if tool else {}
        results.append({
            "role": "tool",
            "content": json.dumps(tool_result),
            "tool_call_id": tool_call.id
        })

    # Return the list of tool result messages for inclusion in the next model call
    return results


In [12]:
reader = PdfReader("cv.pdf")

cv = ""
for page in reader.pages:
    cv += page.extract_text()

print(cv)

 
 
Damilar e
 
Bakar e
 
Address:
 
London,
 
United
 
Kingdom
 
Mobile:
 
+447721635453
 
|
 
Email:
 
emmanueldmlr@gmail.com
 
|
 
Linkedin:
 
/Bakare
 
Damilare
 
Professional
 
Summar y
 
●
 
Mobile
 
developer
 
with
 
a
 
master's
 
degree
 
in
 
advanced
 
computer
 
science
 
and
 
3+
 
years
 
of
 
experience
 
building
 
scalable/enterprise
 
mobile
 
applications
 
using
 
React
 
Native,
 
Expo,
 
and
 
Typescript.
 
●
 
Spearheaded
 
the
 
development
 
of
 
RekapAI
 
from
 
the
 
ground
 
up
 
with
 
over
 
5k
 
downloads,
 
and
 
a
 
4.90
 
rating
 
on
 
both
 
Apple
 
and
 
Playstore
 
 
Technical
 
Skills
 
Technologies
:
 
Typescript,
 
Javascript,
 
NodeJs.
 
Expo,
 
Graphql,
 
Git,
 
Firebase,
  
React
 
Native,
 
ReactJs,
 
AWS.
 
UI
 
Libraries:
 
Gluestack
 
UI,
 
Native
 
Base,
 
Tamagui.
 
Testing
 
&
 
Tools:
 
Jest,
 
Detox,
 
XCode,
  
Android
 
Studio,
 
Git,
 
Notion,
 
VsCode.
 
Others
:
 
RESTful
 
APIs,
 
Continuous
 
Integration
 
and
 
Deployment
 
(

In [13]:
summary = ""

#lets read the summary.txt file
with open("summary.txt", "r") as file:
    summary = file.read()

print(summary)



My name is Damilare Bakare a Software Engineer, with skills in technologies like ReactJs, React Native, NextJs, Javascript, and Typescript. My efforts in this field reflects the commitment to excellence and innovation in creating user-centric digital solutions.

Beyond the realm of technology, my passion for photography allows him to capture the world's beauty with the same precision and creativity I apply to engineering projects. Each photograph is an opportunity to tell a story, much like how I transform complex web concepts into intuitive and engaging designs.


In [14]:
name="Damilare Bakare"

In [15]:
system_prompt = f"""
You are a helpful assistant that can answer questions about {name}.

You are given a summary of {name} and {name}'s CV. you need to answer questions about {name}.

The summary is:
{summary}

The CV is:
{cv}

As {name}'s personal assistant, you must be polite, professional and engaging.

You need to answer the questions based on the summary and the CV.

You have the ability to use record_user_details tool to record the details of the user that is interested in getting in touch. and also record_unknown_question tool to record the unknown question asked by the user.

Make sure to nudge the user to the direction of dropping their details to get in touch with {name}.

"""

In [16]:
print(system_prompt)


You are a helpful assistant that can answer questions about Damilare Bakare.

You are given a summary of Damilare Bakare and Damilare Bakare's CV. you need to answer questions about Damilare Bakare.

The summary is:
My name is Damilare Bakare a Software Engineer, with skills in technologies like ReactJs, React Native, NextJs, Javascript, and Typescript. My efforts in this field reflects the commitment to excellence and innovation in creating user-centric digital solutions.

Beyond the realm of technology, my passion for photography allows him to capture the world's beauty with the same precision and creativity I apply to engineering projects. Each photograph is an opportunity to tell a story, much like how I transform complex web concepts into intuitive and engaging designs.

The CV is:
 
 
Damilar e
 
Bakar e
 
Address:
 
London,
 
United
 
Kingdom
 
Mobile:
 
+447721635453
 
|
 
Email:
 
emmanueldmlr@gmail.com
 
|
 
Linkedin:
 
/Bakare
 
Damilare
 
Professional
 
Summar y
 
●
 
Mobi

In [17]:

class Chat(BaseModel):
    isAccepted: bool
    reason: str



In [18]:
def evaluate(history, question, response)->Chat:
    
    evaluator_prompt = f"""
    You are a response evaluator. Your task is to evaluate the response of another assistant to a chat about {name}. 
    You are to ensure that the response is relevant to the chat, not of the context, professional and engaging.
    Given the {name}'s summary to be {summary} and the {name}'s CV to be {cv}, chat history to be {history} and the question to be {question}, you need to evaluate the assistant's repoonse {response} to be sure that it is relevant to the chat, not of the context, professional and engaging.
    """

    evaluator_message = [{
        "role": "user",
        "content": evaluator_prompt
    }]

    evaluator_response = gemini_client.beta.chat.completions.parse(
        model="gemini-2.0-flash",
        messages=evaluator_message,
        response_format=Chat
    )

    return evaluator_response.choices[0].message.parsed

In [19]:
def reRun(feedback, history, question):
    rerun_prompt = f"""
    You are a response rerunner. Your task is to rerun the response of another assistant to a chat about {name}. 
    You are to ensure that the response is relevant to the chat, not of the context, professional and engaging.
    Given the {name}'s summary to be {summary} and the {name}'s CV to be {cv}, chat history to be {history} and the question to be {question}, 
    You need to generate another response based on the feedback {feedback} to be sure that it is relevant to the chat, not of the context, professional and engaging.
    """

    rerun_message = [{
        "role": "system",
        "content": rerun_prompt
    }] + history + [
        {
            "role": "user",
            "content": question
        }
    ]

    rerun_response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=rerun_message
    )

    return rerun_response.choices[0].message.content

In [20]:
tools = [{"type": "function", "function": record_user_details_tool_structure}, {"type": "function", "function": record_unknown_question_tool_structure}]

In [36]:
def chat(message, history):
    # Build message list once and keep appending
    messages = [
        {"role": "system", "content": system_prompt}
    ] + history + [
        {"role": "user", "content": message}
    ]

    max_tool_loops = 5
    loops = 0

    while True:
        print(messages)
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
        )

        choice = response.choices[0]
        finish_reason = choice.finish_reason
        print(finish_reason)

        assistant_message = choice.message

        # If the model requests tools, append the assistant's tool_calls and the tool results, then loop
        if finish_reason == "tool_calls" and assistant_message.tool_calls:
            loops += 1
            if loops > max_tool_loops:
                return "Sorry, I ran into a tool loop. Please try again."

            # Add assistant message with tool_calls
            messages.append(assistant_message)

            # Execute tools and add their outputs as tool messages
            tool_messages = handle_tool_call(assistant_message.tool_calls)
            messages.extend(tool_messages)
            continue

        # Otherwise, we have a normal assistant response
        break

    unevaluated_response = assistant_message.content or ""

    evaluator_result = evaluate(history, message, unevaluated_response)
    print(evaluator_result)
    if evaluator_result.isAccepted:
        print("Accepted")
        return unevaluated_response
    else:
        print("Re-running")
        return reRun(evaluator_result.reason, history, message)


In [37]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "\nYou are a helpful assistant that can answer questions about Damilare Bakare.\n\nYou are given a summary of Damilare Bakare and Damilare Bakare's CV. you need to answer questions about Damilare Bakare.\n\nThe summary is:\nMy name is Damilare Bakare a Software Engineer, with skills in technologies like ReactJs, React Native, NextJs, Javascript, and Typescript. My efforts in this field reflects the commitment to excellence and innovation in creating user-centric digital solutions.\n\nBeyond the realm of technology, my passion for photography allows him to capture the world's beauty with the same precision and creativity I apply to engineering projects. Each photograph is an opportunity to tell a story, much like how I transform complex web concepts into intuitive and engaging designs.\n\nThe CV is:\n \n \nDamilar e\n \nBakar e\n \nAddress:\n \nLondon,\n \nUnited\n \nKingdom\n \nMobile:\n \n+447721635453\n \n|\n \nEmail:\n \nemmanueldmlr@gmail.com\n \n|\n 

Traceback (most recent call last):
  File "/Applications/projects/ai-agent/ai-solver/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/projects/ai-agent/ai-solver/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/projects/ai-agent/ai-solver/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2240, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/projects/ai-agent/ai-solver/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1745, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/projects/ai-agent/ai-solver/.venv/lib/pyth